In [10]:
from fredapi import Fred
import pandas as pd
import plotly.graph_objects as go
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

fred = Fred(api_key="d0b19bd01f39fdd7318477768791c1a9")

states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD',
          'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
          'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

# Dictionary to hold the data for each series
df = pd.DataFrame(columns=states)

# Loop through each ticker and fetch the series data
for x in states:
    try:
        df[x] = fred.get_series(x + 'UR')
    except Exception as e:
        print(f"Error fetching data for {x + 'UR'}: {e}")

df_sahm = df.rolling(3).mean() - df.rolling(3).mean().rolling(12).min()
df_sahm_count = df_sahm.applymap(lambda x: 1 if x > 0.5 else 0)
sahm_series = df_sahm_count.sum(axis=1).rename('Number of US States Breaching Sahm Rule Recession Threshold')

# Fetch US recession data
recession_data = fred.get_series('USREC')
recession_periods = recession_data[recession_data == 1]
recession_periods = recession_periods[recession_periods.index >= sahm_series.index[0]]

fig = go.Figure()

# Add Sahm series to the plot
fig.add_trace(go.Scatter(x=sahm_series.index, y=sahm_series.values, mode='lines', name='Number of US States Breaching Sahm Rule'))

# Identify recession periods and add shading
in_recession = False
for date in recession_periods.index:
    if not in_recession:
        recession_start = date
        in_recession = True
    if in_recession and (date + pd.DateOffset(months=1) not in recession_periods.index):
        fig.add_shape(type="rect",
                      x0=recession_start, x1=date + pd.DateOffset(months=1),
                      y0=0, y1=sahm_series.max(),
                      fillcolor="gray", opacity=0.2,
                      layer="below", line_width=0)
        in_recession = False

# Update layout
fig.update_layout(title="Number of US States Breaching Sahm Rule Recession Threshold",
                  xaxis_title="Date",
                  yaxis_title="Number of States",
                  showlegend=True)

fig.show()

In [5]:
recession_data = fred.get_series('USREC')
recession_periods = recession_data[recession_data == 1].index

In [7]:
sahm_series.index

DatetimeIndex(['1976-01-01', '1976-02-01', '1976-03-01', '1976-04-01',
               '1976-05-01', '1976-06-01', '1976-07-01', '1976-08-01',
               '1976-09-01', '1976-10-01',
               ...
               '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01',
               '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01'],
              dtype='datetime64[ns]', length=582, freq=None)